In [ ]:
from pickle import dump, load
from transformers import BlipProcessor, BlipForConditionalGeneration
import torch
from PIL import Image
import matplotlib.pyplot as plt
import os

DESCARGAR EL MODELO Y EL TOKENIZADOR

In [ ]:
# Cargar el procesador y el modelo
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

# Guardar el procesador y el modelo localmente en las rutas especificadas
# MODIFICAR LAS RUTAS SEGÚN CADA UNO
processor_path = "E:\\SATURDAYSAI\\PROYECTO_FINAL\\modelos_preentrenados\\blip-image-captioning-processor"
model_path = "E:\\SATURDAYSAI\\PROYECTO_FINAL\\modelos_preentrenados\\blip-image-captioning-model"

processor.save_pretrained(processor_path)
model.save_pretrained(model_path)

# Función para calcular el tamaño del directorio
def get_directory_size(directory):
    total_size = 0
    for dirpath, dirnames, filenames in os.walk(directory):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            total_size += os.path.getsize(fp)
    return total_size

# Verificar el tamaño de los directorios guardados
processor_size = get_directory_size(processor_path) / (1024 * 1024)  # Convertir a MB
model_size = get_directory_size(model_path) / (1024 * 1024)  # Convertir a MB

print(f"Tamaño del procesador: {processor_size:.2f} MB")
print(f"Tamaño del modelo: {model_size:.2f} MB")

In [ ]:

#Verificar si existen nucleos cuda
print(f"CUDA disponible: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"Nombre del dispositivo: {torch.cuda.get_device_name(0)}")

# Configurar el dispositivo (GPU o CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "gpu")
model.to(device)


PREPROCESAR LA IMAGEN

In [ ]:
# Lista de rutas de las imágenes
image_paths = [
    "data\input\imagenes_prueba\\000000101762.jpg",
    "data\input\imagenes_prueba\\000000001761.jpg",
    "data\output\divide_images\panel_0.jpg"
]

In [ ]:
# Función para cargar y preprocesar una imagen
def load_and_preprocess_image(image_path):
    image = Image.open(image_path).convert("RGB")
    inputs = processor(images=image, return_tensors="pt").to(device)
    return inputs, image

GENERAR LA DESCRIPCIÓN DE LA IMAGEN

In [ ]:
# Bucle para describir cada imagen
for image_path in image_paths:
    inputs, image = load_and_preprocess_image(image_path)
    # Generar la descripción de la imagen con parámetros ajustados
    outputs = model.generate(
        **inputs,
        max_length=100,  # Longitud máxima de la descripción
        num_beams=5,    # Búsqueda en haz
        length_penalty=2.0,  # Penalización por longitud
        early_stopping=True
    )
    caption = processor.decode(outputs[0], skip_special_tokens=True)
    
    # Imprimir la descripción y mostrar la imagen
    print(f"Descripción de {image_path}: {caption}")
    plt.imshow(image)
    plt.title(caption)
    plt.axis('off')
    plt.show()
